In [3]:
# import packages
import pandas as pd
import pymysql

# Establish connection
mydb = pymysql.connect(
    host="localhost",
    user="root",
    password="nITHIY@1234"
)

print(mydb)

# Create cursor
Mycursor = mydb.cursor()  # Cursor is the interaction point between MySQL and Python

# Create database
Mycursor.execute('CREATE DATABASE SportRadar')
print('The database has been created')

# Show databases
Mycursor.execute('SHOW DATABASES')
for i in Mycursor:
    print(''.join(i))

# Close the connection
mydb.close()

The database has been created
information_schema
mysql
performance_schema
record
sakila
sportradar
stduent
sys
world


In [11]:
import pymysql
import pandas as pd

# Establishing a connection to the database
mydb = pymysql.connect(
    host="localhost",
    user="root",
    password="nITHIY@1234",
    database="SportRadar"  # Ensure you're connecting to the correct database
)

# Creating a cursor object
Mycursor = mydb.cursor()

# Import CSV file into a DataFrame
df = pd.read_csv(r'C:\Users\Nithiyaasri G\python projects\Mini_Project\venv\Scripts\table_2.csv')
Categories_table = df.copy()

# Data preprocessing
print(Categories_table.columns)  # Check the column names to ensure they are correct

# Drop duplicates based on 'category_id'
Categories_table.drop_duplicates(subset=['category_id'], inplace=True)

# Create table (use IF NOT EXISTS to avoid errors if the table already exists)
create_table_query = '''
CREATE TABLE IF NOT EXISTS SportRadar.Categories_table (
    category_id VARCHAR(50),
    category_name VARCHAR(100) NOT NULL,
    PRIMARY KEY (category_id)
);
'''
Mycursor.execute(create_table_query)
print('Table has been created (or already exists).')

# Check if the DataFrame has the correct columns
if 'category_id' not in Categories_table.columns or 'category_name' not in Categories_table.columns:
    print("Error: DataFrame does not have 'category_id' or 'category_name' columns.")
else:
    # Values to store
    insert_into = 'INSERT IGNORE INTO SportRadar.Categories_table (category_id, category_name) VALUES (%s, %s)'

    # Insert the rows through the loop
    for index, row in Categories_table.iterrows():
        try:
            # Ensure column names are correct
            Mycursor.execute(insert_into, (row['category_id'], row['category_name']))
        except Exception as e:
            print(f"Error inserting row {index}: {row.to_dict()}\nError: {e}")

    # Commit the transaction after all rows are inserted
    mydb.commit()

    # Ensure the data has been stored
    Mycursor.execute('SELECT COUNT(*) FROM SportRadar.Categories_table')
    row_count = Mycursor.fetchone()[0]
    print(f'The data has been uploaded. Total rows in table: {row_count}')

# Close the cursor and connection
Mycursor.close()
mydb.close()


Index(['category_id', 'category_name'], dtype='object')
Table has been created (or already exists).
The data has been uploaded. Total rows in table: 18


In [17]:
import pymysql
import pandas as pd

# Establishing a connection to the database
mydb = pymysql.connect(
    host="localhost",
    user="root",
    password="nITHIY@1234",
    database="SportRadar"  # Ensure you're connecting to the correct database
)

# Creating a cursor object
Mycursor = mydb.cursor()

# Import file to store
df = pd.read_csv(r'C:\Users\Nithiyaasri G\python projects\Mini_Project\venv\Scripts\table_1.csv')
Competitions_table = df.copy()

# Data Preprocessing

# Check column names to make sure 'competition_id' exists
print("Columns in the dataset:", Competitions_table.columns)

# If 'competition_id' is missing, generate it as before
if 'competition_id' not in Competitions_table.columns:
    Competitions_table['competition_id'] = 'comp_' + Competitions_table.index.astype(str)

# Drop duplicates by 'competition_id'
Competitions_table.drop_duplicates(subset=['competition_id'], inplace=True)

# Convert missing values in 'parent_id' to None (for MySQL compatibility)
Competitions_table['parent_id'] = Competitions_table['parent_id'].apply(lambda x: None if pd.isna(x) else x)

# Check the cleaned DataFrame
print(Competitions_table.head())

# Create table in MySQL (if not exists)
Mycursor.execute('''
    CREATE TABLE IF NOT EXISTS SportRadar.Competitions_table (
        competition_id VARCHAR(50), 
        competition_name VARCHAR(100) NOT NULL,
        parent_id VARCHAR(50) NULL,
        type VARCHAR(20) NOT NULL,
        gender VARCHAR(10) NOT NULL,
        category_id VARCHAR(50),
        PRIMARY KEY(competition_id),
        FOREIGN KEY (category_id) REFERENCES SportRadar.Categories_table(category_id)
    )
''')

# Prepare the insert statement
insert_into = '''
    INSERT IGNORE INTO SportRadar.Competitions_table 
    (competition_id, competition_name, parent_id, type, gender, category_id) 
    VALUES (%s, %s, %s, %s, %s, %s)
'''

# Insert the rows through the loop
for index, row in Competitions_table.iterrows():
    try:
        # Corrected mapping for the columns
        Mycursor.execute(insert_into, (
            row['competition_id'], 
            row['name'],  # This is the correct column name for competition_name
            row['parent_id'], 
            row['type'], 
            row['gender'], 
            row['category_id']
        ))
    except Exception as e:
        print(f"Error inserting row {index}: {row.to_dict()}\nError: {e}")

# Commit the transaction
mydb.commit()

# Ensure the data has been stored
Mycursor.execute('SELECT COUNT(*) FROM SportRadar.Competitions_table')
row_count = Mycursor.fetchone()[0]
print(f'The data has been uploaded. Total rows in table: {row_count}')


Columns in the dataset: Index(['id', 'name', 'type', 'gender', 'category_id', 'parent_id'], dtype='object')
                    id                               name     type gender  \
0   sr:competition:620                         Hopman Cup    mixed  mixed   
1   sr:competition:660                     World Team Cup    mixed    men   
2   sr:competition:990         ATP Challenger Tour Finals  singles    men   
3  sr:competition:1207  Championship International Series  singles  women   
4  sr:competition:2100                          Davis Cup    mixed    men   

       category_id            parent_id competition_id  
0  sr:category:181                  NOT         comp_0  
1    sr:category:3                  NOT         comp_1  
2   sr:category:72  sr:competition:6239         comp_2  
3    sr:category:6                  NOT         comp_3  
4   sr:category:76                  NOT         comp_4  
The data has been uploaded. Total rows in table: 5842


In [18]:
# import file to store
df=pd.read_csv(r'C:\Users\Nithiyaasri G\python projects\Mini_Project\venv\Scripts\table_3.csv')
Complexes_table=df.copy()

# Data Preprocessing

# information about dataset
Complexes_table.info()
# drop duplicates
Complexes_table.drop_duplicates(subset=['complex_id'],inplace=True)


# create table 
Mycursor.execute('create table SportRadar.Complexes_table(complex_id varchar(50),complex_name varchar(100) not null,primary key(complex_id))')
print('the table has been created')

# values to store 
insert_into='insert ignore into SportRadar.Complexes_table(complex_id,complex_name)values(%s,%s)'
#insert the rows through the loop
for index,row in Complexes_table.iterrows():
    try:
        Mycursor.execute(insert_into,(row['complex_id'],row['complex_name']))
    except Exception as e:
        print(f"Error inserting row {index}: {row.to_dict()}\nError: {e}")
#commit changes        
mydb.commit()

# ensure the data has been stored
Mycursor.execute('select count(*) from SportRadar.Complexes_table')
row_count=Mycursor.fetchone()[0]
print(f'The data has been uploaded. Total rows in table: {row_count}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   complex_id    617 non-null    object
 1   complex_name  617 non-null    object
dtypes: object(2)
memory usage: 9.8+ KB
the table has been created
The data has been uploaded. Total rows in table: 617


In [24]:
# import file to store
df=pd.read_csv(r'C:\Users\Nithiyaasri G\python projects\Mini_Project\venv\Scripts\table_4.csv')
Venues_table=df.copy()

# Data Preprocessing

#informtion about the dataset
Venues_table.info()
#check null values by related columns
missing=Venues_table.loc[Venues_table['timezone'].isna(),['country_name','city_name','timezone']]
missing.drop_duplicates()
# mapping the missing timezone
fill_missing = {('France', 'Orchies'): 'Europe/Paris',('Singapore', 'Kallang'): 'Asia/Singapore',('China', 'Zhuhai'): 'Asia/Shanghai',('Russia', 'Moscow'): 'Europe/Moscow',('Chile', 'Vina del Mar'): 'America/Santiago',('China', 'Liuzhou'): 'Asia/Shanghai',('Uruguay', 'Punta del Este'): 'America/Montevideo',('Austria', 'Tulln'): 'Europe/Vienna',('Monaco', 'Monte Carlo'): 'Europe/Paris',('Germany', 'Baden-Wurttemberg'): 'Europe/Berlin',('Switzerland', 'Montreux'): 'Europe/Zurich',('China', 'Jiujiang'): 'Asia/Shanghai',('France', 'Decines-Charpieu'): 'Europe/Paris',('Thailand','Nonthaburi'):'Asia/Bangkok'}
# fill missing values by 'combine key' column
Venues_table['combined_key'] = list(zip(Venues_table['country_name'], Venues_table['city_name']))
Venues_table['timezone'] = Venues_table['combined_key'].map(fill_missing).fillna(Venues_table['timezone'])
#drop the column
Venues_table.drop(columns=['combined_key'], inplace=True)
# check null values are there  
print(Venues_table.isna().sum())



# create table 
Mycursor.execute('create table SportRadar.Venues_table(venue_id varchar(50),venue_name varchar(100) not null,city_name varchar(100) not null,country_name varchar(100) not null,country_code char(3) not null,timezone varchar(100) not null,complex_id varchar(50),primary key(venue_id),foreign key (complex_id) references SportRadar.Complexes_table(complex_id))')
print('the table has been created')

# values to store 
insert_into='insert ignore into SportRadar.Venues_table(venue_id,venue_name,city_name,country_name,country_code,timezone,complex_id)values(%s,%s,%s,%s,%s,%s,%s)'

#insert the rows through the loop
for index,row in Venues_table.iterrows():
    try:
        Mycursor.execute(insert_into,(row['venue_id'],row['venue_name'],row['city_name'],row['country_name'],row['country_code'],row['timezone'],row['complex_id']))
    except Exception as e:
        print(f"Error inserting row {index}: {row.to_dict()}\nError: {e}")

#commit changes
mydb.commit()

# ensure the data has been stored
Mycursor.execute('select count(*) from SportRadar.Venues_table')
row_count=Mycursor.fetchone()[0]
print(f'The data has been uploaded. Total rows in table: {row_count}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3187 entries, 0 to 3186
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   complex_id    3187 non-null   object
 1   venue_id      3187 non-null   object
 2   venue_name    3187 non-null   object
 3   city_name     3187 non-null   object
 4   country_name  3187 non-null   object
 5   country_code  3187 non-null   object
 6   timezone      3187 non-null   object
dtypes: object(7)
memory usage: 174.4+ KB
complex_id      0
venue_id        0
venue_name      0
city_name       0
country_name    0
country_code    0
timezone        0
dtype: int64
the table has been created
The data has been uploaded. Total rows in table: 3187


In [18]:
import pandas as pd
import pymysql

# Assuming you have established a connection to your MySQL database
# mydb = pymysql.connect(...)
# Mycursor = mydb.cursor()

# Import the CSV file
df = pd.read_csv(r'C:\Users\Nithiyaasri G\python projects\Mini_Project\venv\Scripts\table_5.csv')
Competitor_Rankings_table = df.copy()

# Data Preprocessing

# Information about dataset
Competitor_Rankings_table.info()

# Drop duplicates based on the 'rank' column (update column name if needed)
Competitor_Rankings_table.drop_duplicates(subset=['rank'], inplace=True)

# Check for null values
print(Competitor_Rankings_table.isna().sum())

# Create table if it doesn't already exist
create_table_query = '''
CREATE TABLE IF NOT EXISTS SportRadar.Competitor_Rankings_table (
    rank_id INT AUTO_INCREMENT,
    `rank` INT NOT NULL,
    movement INT NOT NULL,
    points INT NOT NULL,
    competitions_played INT NOT NULL,
    competitor_id VARCHAR(50),
    PRIMARY KEY (rank_id),
    FOREIGN KEY (competitor_id) REFERENCES SportRadar.competitors_table(competitor_id)
);
'''
Mycursor.execute(create_table_query)
print('The table has been created (or already exists).')

# Prepare the insert query
insert_into = '''INSERT IGNORE INTO SportRadar.Competitor_Rankings_table
                 (`rank`, movement, points, competitions_played, competitor_id)
                 VALUES (%s, %s, %s, %s, %s)'''

# Convert dataframe to list of tuples for batch insertion
data_to_insert = Competitor_Rankings_table[['rank', 'movement', 'points', 'competitions_played', 'competitor_id']].values.tolist()

# Insert data in bulk using executemany for better performance
try:
    Mycursor.executemany(insert_into, data_to_insert)
    mydb.commit()
    print("Data has been uploaded successfully.")
except Exception as e:
    print(f"Error during bulk insert: {e}")

# Ensure the data has been stored
Mycursor.execute('SELECT COUNT(*) FROM SportRadar.Competitor_Rankings_table')
row_count = Mycursor.fetchone()[0]
print(f'The data has been uploaded. Total rows in table: {row_count}')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   rank                 1000 non-null   int64 
 1   movement             1000 non-null   int64 
 2   points               1000 non-null   int64 
 3   competitions_played  1000 non-null   int64 
 4   competitor_id        1000 non-null   object
dtypes: int64(4), object(1)
memory usage: 39.2+ KB
rank                   0
movement               0
points                 0
competitions_played    0
competitor_id          0
dtype: int64
The table has been created (or already exists).
Data has been uploaded successfully.
The data has been uploaded. Total rows in table: 502


In [17]:
# import file to store
df=pd.read_csv(r'C:\Users\Nithiyaasri G\python projects\Mini_Project\venv\Scripts\table_6.csv')
competitors_table=df.copy()

# Data Preprocessing

# information about dataet
competitors_table.info()
# Drop duplicate rows based on 'competitor_id'
competitors_table.drop_duplicates(subset=['competitor_id'],inplace=True)
# check null values
competitors_table.loc[competitors_table['country_code'].isna(),'country']
competitors_table[competitors_table['country']=='Neutral']
competitors_table['country_code'].fillna('NTL',inplace=True)
# ensure the null values
print(competitors_table.isna().sum())


# create table 
Mycursor.execute('create table SportRadar.competitors_table(competitor_id varchar(50),name varchar(100) not null,country varchar(100) not null,country_code char(3) not null,abbreviation varchar(10) not null,primary key (competitor_id))')
print('the table has been created')
# values to store 
insert_into='insert ignore into SportRadar.competitors_table(competitor_id,name,country,country_code,abbreviation)values(%s,%s,%s,%s,%s)'

#insert the rows through the loop
for index,row in competitors_table.iterrows():
    try:
        Mycursor.execute(insert_into,(row['competitor_id'],row['name'],row['country'],row['country_code'],row['abbreviation']))
    except Exception as e:
        print(f"Error inserting row {index}: {row.to_dict()}\nError: {e}")

# committ chanes        
mydb.commit()

# ensure the data has been stored
Mycursor.execute('select count(*) from SportRadar.competitors_table')
row_count=Mycursor.fetchone()[0]
print(f'The data has been uploaded. Total rows in table: {row_count}')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   competitor_id  1000 non-null   object
 1   name           1000 non-null   object
 2   country        1000 non-null   object
 3   country_code   936 non-null    object
 4   abbreviation   1000 non-null   object
dtypes: object(5)
memory usage: 39.2+ KB
competitor_id    0
name             0
country          0
country_code     0
abbreviation     0
dtype: int64


C:\Users\Nithiyaasri G\AppData\Local\Temp\ipykernel_17156\1383650677.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  competitors_table['country_code'].fillna('NTL',inplace=True)


the table has been created
The data has been uploaded. Total rows in table: 1000
